In [1]:
# Test Deployed Agent
# New Session
from dotenv import load_dotenv
import os
from vertexai import agent_engines
import uuid
from utils.utils import *

load_dotenv()

# Engine Models loaded from .env
crm_stage_agent = os.getenv("CRM_STAGE_AGENT")
omni_stage_agent = os.getenv("OMNI_STAGE_AGENT")

# Get a handle to the existing engine
remote_app = agent_engines.get(crm_stage_agent)


Credenciales: None
✅ Conectado al bucket: adk_hackathon_bucket


In [2]:
# Initialize crm_stage_agent Engine. No initial session required.
u_id = f"u_{uuid.uuid4().hex[:8]}"
print(f'Session ID: {u_id}')

remote_session = remote_app.create_session(
    user_id=u_id,
    #state=ready_state # Comment/Uncomment if session requires initial state
    )
print("Session:", remote_session["id"])

Session ID: u_54e75a0c
Session: 190976923407482880


In [9]:
# Interact with CRM Stage Agent
user_query_test = input('YOU: ')
for event in remote_app.stream_query(
    user_id= remote_session["userId"],
    session_id= remote_session["id"],
    message=user_query_test,
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-bdce3b4e-6df5-4964-acdc-4fd64e508dea', 'args': {'agent_name': 'sequential_agent'}, 'name': 'transfer_to_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 11, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 11}], 'prompt_token_count': 1468, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 1468}], 'total_token_count': 1479, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-98750d75-d51b-46b9-9eed-a5be50fac217', 'author': 'intake_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'TIDf9zCn', 'timestamp': 1750551332.550369}
{'content': {'parts': [{'function_response': {'id': 'adk-bdce3b4e-6df5-4964-acdc-4fd64e508dea', 'name': 'transfer_to_agent', 'response': {'result': None}}}], 'role': 'user'}, 'invocation_id': 'e-98750d75-d51b-46b9-9eed-a5be50fac217', 'author': 'intake_agent', 'actions': {'state_delta': {},

In [7]:
# UPLOAD FILE FUNCION
# Use it if the user uploads a document, to add it to the data bucket in GCS
#from utils.utils import *
local_file = "/Users/alex/Projects/Auto Flujo/Apps/AI Agents/hackathon_data/Medexa Dental Profile.pdf"


await handle_upload_and_patch_state(crm_stage_agent,
                                    local_file,                                     
                                    user_id= remote_session["userId"], 
                                    session_id= remote_session["id"], 
                                    )

✅ Subido: /Users/alex/Projects/Auto Flujo/Apps/AI Agents/hackathon_data/Medexa Dental Profile.pdf → gs://adk_hackathon_bucket/crm_demo/Medexa Dental Profile.pdf
Creating new RAG Corpus for u_490ee23f...
Successfully created corpus: projects/49793268080/locations/us-central1/ragCorpora/3819052484010180608
Started import of gs://adk_hackathon_bucket/crm_demo/Medexa Dental Profile.pdf into corpus projects/49793268080/locations/us-central1/ragCorpora/3819052484010180608. Response: imported_rag_files_count: 1

✅ Session 1871945484323520512 patched with uploads & rag_corpus


In [10]:
# The backend system must check if the pipeline is completed before continuing with the CRM building process
remote_session_state = remote_app.get_session(
    user_id = remote_session["userId"],
    session_id = remote_session["id"],
)

intake_state = remote_session_state['state']
is_pipeline_complete = remote_session_state.get('state', {}).get('pipeline', {}).get('pipeline_completed',False)

print(f"Top-level keys in state: {list(intake_state.keys())}")
print(f'Pipeline Completed Status: {is_pipeline_complete}')

# If the pipeline is completed, it will return True, otherwise False.

Top-level keys in state: ['pipeline_assembler_agent', 'stage_designer_agent_response', 'intake_completed', 'field_tag_selector_agent_response', 'pipeline', 'stage_agent_creator_agent_response']
Pipeline Completed Status: True


In [14]:
intake_state

{'pipeline_assembler_agent': 'The CRM pipeline has been successfully assembled and is now complete and ready to use.\n',
 'stage_designer_agent_response': '```json\n[\n  {\n    "stage_number": 1,\n    "stage_name": "Lead Qualification",\n    "entry_condition": "A new lead is received through any channel (website, inquiry, etc.).",\n    "brief_stage_goal": "Determine if the lead is genuinely interested in buying property and meets initial qualification criteria."\n  },\n  {\n    "stage_number": 2,\n    "stage_name": "Appointment Scheduling",\n    "entry_condition": "Lead is qualified and expresses interest in viewing properties.",\n    "brief_stage_goal": "Schedule property viewings and appointments for qualified leads with available agents."\n  },\n  {\n    "stage_number": 3,\n    "stage_name": "Property Viewing",\n    "entry_condition": "An appointment has been scheduled and completed.",\n    "brief_stage_goal": "The lead attends a viewing, after which the agent collects feedback and 

In [18]:
def print_all_keys(obj, parent_key=''):
    if isinstance(obj, dict):
        for k, v in obj.items():
            full_key = f"{parent_key}.{k}" if parent_key else k
            print_all_keys(v, full_key)
    elif isinstance(obj, list):
        for idx, item in enumerate(obj):
            full_key = f"{parent_key}[{idx}]"
            print_all_keys(item, full_key)
    else:
        print(parent_key)

print_all_keys(intake_state)


rag_corpus
uploaded_docs[0].gcs_uri
uploaded_docs[0].filename
stage_designer_agent_response
pipeline_assembler_agent
pipeline.pipeline_completed
pipeline.stage_design_results.stages[0].brief_stage_goal
pipeline.stage_design_results.stages[0].prompt
pipeline.stage_design_results.stages[0].user_tags[0]
pipeline.stage_design_results.stages[0].stage_name
pipeline.stage_design_results.stages[0].entry_condition
pipeline.stage_design_results.stages[0].fields[0]
pipeline.stage_design_results.stages[0].fields[1]
pipeline.stage_design_results.stages[0].fields[2]
pipeline.stage_design_results.stages[0].fields[3]
pipeline.stage_design_results.stages[0].fields[4]
pipeline.stage_design_results.stages[0].fields[5]
pipeline.stage_design_results.stages[0].stage_number
pipeline.stage_design_results.stages[1].brief_stage_goal
pipeline.stage_design_results.stages[1].prompt
pipeline.stage_design_results.stages[1].user_tags[0]
pipeline.stage_design_results.stages[1].stage_name
pipeline.stage_design_results.

In [15]:
intake_state.keys()

dict_keys(['pipeline_assembler_agent', 'stage_designer_agent_response', 'intake_completed', 'field_tag_selector_agent_response', 'pipeline', 'stage_agent_creator_agent_response'])

In [19]:
# Prepar initial state for Omni Stage Agent
ready_state = await build_ready_state(session_state=remote_session_state['state'],current_stage=1)

In [21]:
ready_state.keys()

dict_keys(['intake_completed', 'uploaded_docs', 'rag_corpus', 'pipeline_completed', 'business_id', 'biz_name', 'biz_info', 'goal', 'kb_files', 'total_stages', 'stage_1_brief_stage_goal', 'stage_1_prompt', 'stage_1_user_tags', 'stage_1_stage_name', 'stage_1_entry_condition', 'stage_1_fields', 'stage_1_stage_number', 'stage_2_brief_stage_goal', 'stage_2_prompt', 'stage_2_user_tags', 'stage_2_stage_name', 'stage_2_entry_condition', 'stage_2_fields', 'stage_2_stage_number', 'stage_3_prompt', 'stage_3_brief_stage_goal', 'stage_3_user_tags', 'stage_3_stage_name', 'stage_3_entry_condition', 'stage_3_fields', 'stage_3_stage_number', 'stage_4_brief_stage_goal', 'stage_4_prompt', 'stage_4_user_tags', 'stage_4_stage_name', 'stage_4_entry_condition', 'stage_4_fields', 'stage_4_stage_number', 'current_stage', 'current_artifacts', 'current_stage_name', 'current_stage_brief_goal', 'current_stage_prompt', 'current_stage_fields', 'current_stage_user_tags', 'all_stages_names_and_descriptions_and_entry_c

In [22]:
print_all_keys(ready_state)

intake_completed
uploaded_docs[0].gcs_uri
uploaded_docs[0].filename
rag_corpus
pipeline_completed
business_id
biz_name
biz_info
goal
kb_files[0]
total_stages
stage_1_brief_stage_goal
stage_1_prompt
stage_1_user_tags[0]
stage_1_stage_name
stage_1_entry_condition
stage_1_fields[0]
stage_1_fields[1]
stage_1_fields[2]
stage_1_fields[3]
stage_1_fields[4]
stage_1_fields[5]
stage_1_stage_number
stage_2_brief_stage_goal
stage_2_prompt
stage_2_user_tags[0]
stage_2_stage_name
stage_2_entry_condition
stage_2_fields[0]
stage_2_fields[1]
stage_2_fields[2]
stage_2_fields[3]
stage_2_fields[4]
stage_2_fields[5]
stage_2_stage_number
stage_3_prompt
stage_3_brief_stage_goal
stage_3_user_tags[0]
stage_3_stage_name
stage_3_entry_condition
stage_3_fields[0]
stage_3_fields[1]
stage_3_fields[2]
stage_3_fields[3]
stage_3_fields[4]
stage_3_stage_number
stage_4_brief_stage_goal
stage_4_prompt
stage_4_user_tags[0]
stage_4_stage_name
stage_4_entry_condition
stage_4_fields[0]
stage_4_fields[1]
stage_4_fields[2]
sta

In [ ]:
# Initialize Omni Stage Agent session with final user/client
final_remote_app = agent_engines.get(omni_stage_agent)

u_id = f"u_{uuid.uuid4().hex[:8]}"
print(f'Session ID: {u_id}')

final_remote_session = final_remote_app.create_session(
    user_id=u_id,
    state=ready_state
    )
print("Session:", final_remote_session["id"])

In [ ]:
# Interact with Omni Stage Agent
user_query_test = input('YOU: ')
for event in final_remote_app.stream_query(
    user_id= final_remote_session["userId"],
    session_id= final_remote_session["id"],
    message=user_query_test,
):
    print(event)

In [ ]:
# Get the latest session state (it may update with every interaction).
current_session = final_remote_app.get_session(
    user_id = final_remote_session["userId"],
    session_id = final_remote_session["id"],)
#current_session

In [ ]:
# Extract relevant data from the current session state.

# The following fields (user_name to stage) are dynamic and will change with each user.
# They are typically stored in the user database to track individual lead information.
state = current_session.get('state', {})

# Dynamic user data for user database
user_name    = state.get("Name", "")
user_tag     = state.get("Type", "")
company      = state.get("Company", "")
website      = state.get("Website", "")
phone        = state.get("Phone", "")
email        = state.get("Email", "")
address      = state.get("Address", "")
requirements = state.get("Requirements", "")
notes        = state.get("Notes", "")
stage        = state.get("current_stage", 1.0)

# The following fields are static and define business-related context.
# These are shared across all users for the same business and usually do not change.
# Business Data
biz_name    = state.get("biz_name", "")
biz_info    = state.get("biz_info", "")
goal        = state.get("goal", "")
business_id               = state.get("business_id", "")

# Pipeline-level flags, usually not personalized per user
pipeline_completed        = state.get("pipeline_completed", False)
intake_completed          = state.get("intake_completed", False)
total_stages              = state.get("total_stages", "")

# Dynamically extract all stage definitions based on total_stages.
# These are static definitions for each CRM stage.
total_stages_for_funct = int(state.get("total_stages", 0))
stages = []

for i in range(1, total_stages_for_funct + 1):
    stage = {
        "stage_name":         state.get(f"stage_{i}_stage_name", ""),
        "stage_number":       state.get(f"stage_{i}_stage_number", ""),
        "entry_condition":    state.get(f"stage_{i}_entry_condition", ""),
        "prompt":             state.get(f"stage_{i}_prompt", ""),
        "brief_stage_goal":   state.get(f"stage_{i}_brief_stage_goal", ""),
        "fields":             state.get(f"stage_{i}_fields", []),
        "user_tags":          state.get(f"stage_{i}_user_tags", []),
    }
    stages.append(stage)



In [ ]:
# Clean Cloud Resources
remote_app.delete(force=True)
final_remote_app.delete(force=True)